*Internal Note:* 

- *The following is a tutorial template to be used by PO.DAAC staff when developing end-user tutorials.*
- *Copy this template and edit to create your notebook, keeping as many key elements as appropriate.*
- *DO NOT modify this template to create a tutorial. If you wish to make suggestions to improve the template please submit a PR and tag Catalina Oaida Taglialatela (ScienceCat18)*

> From the PO.DAAC Cookbook, to access the GitHub version of the notebook, follow [this link](insert link to notebook).

# SWOT Hydro Science Application Tutorial
#### *Authors: Cerbelaud A., Wade J., Jet Propulsion Laboratory - Caltech*

## Summary

_[Add summary/tutorial description here]_

## Requirements

### 1. Compute environment 
*internal note (remove this note in final tutorial): keep one or both of these Req 1 depending on environment required to run the noteook*

This tutorial can be run in the following environments:
- **AWS instance running in us-west-2**: NASA Earthdata Cloud data in S3 can be directly accessed via temporary credentials; this access is limited to requests made within the US West (Oregon) (code: `us-west-2`) AWS region.
- **Local compute environment** e.g. laptop, server: this tutorial can be run on your local machine

### 2. Earthdata Login

An Earthdata Login account is required to access data, as well as discover restricted data, from the NASA Earthdata system. Thus, to access NASA data, you need Earthdata Login. Please visit https://urs.earthdata.nasa.gov to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.

### 3. Additional Requirements

Any other requirements needed for reproducing this tutorial.

## Learning Objectives
- enter objective 
- enter objective 
- ...

------

## Import Packages

In [1]:
import glob
import os
import requests
import s3fs
import fiona
import netCDF4 as nc
import h5netcdf
import xarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.xarray
import earthaccess
from earthaccess import Auth, DataCollections, DataGranules, Store

## Authenticate
Authenticate your Earthdata Login (EDL) information using the `earthaccess` python package as follows:

In [3]:
auth = earthaccess.login() # Login with your EDL credentials if asked

We are already authenticated with NASA EDL


#### Search for the data of interest

In [4]:
#Retrieves granule from the day we want, in this case by passing to `earthdata.search_data` function the data collection shortname, temporal bounds, and for restricted data one must specify the search count
river_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_RIVERSP_1.1', 
                                        temporal = ('2023-04-08 00:00:00', '2023-04-22 23:59:59'),
                                        granule_name = '*Reach*_009_NA*') # here we filter by Reach files (not node), pass #13 and continent code=NA

Granules found: 15


#### Set up an `s3fs` session for Direct Cloud Access
`s3fs` sessions are used for authenticated access to s3 bucket and allows for typical file-system style operations. Below we create session by passing in the data access information.

In [5]:
fs_s3 = earthaccess.get_s3fs_session(results=river_results)

#### Create Fiona session to work with zip and embedded shapefiles in the AWS Cloud
The native format for this data is a **.zip** file, and we want the **.shp** file within the .zip file, so we will create a Fiona AWS session using the credentials from setting up the s3fs session above to access the shapefiles within the zip files. If we don't do this, the alternative would be to download the data to the cloud environment (e.g. EC2 instance, user S3 bucket) and extract the .zip file there.

In [6]:
fiona_session=fiona.session.AWSSession(
        aws_access_key_id=fs_s3.storage_options["key"],
        aws_secret_access_key=fs_s3.storage_options["secret"],
        aws_session_token=fs_s3.storage_options["token"]
    )

In [7]:
# Initialize list to shp files
SWOT_HR_shps = []

# Loop through queried granules
for j in range(0,len(river_results)):
    
    # Get the link for each zip file
    river_link = earthaccess.results.DataGranule.data_links(river_results[j], access='direct')[0]
    
    # We use the zip+ prefix so fiona knows that we are operating on a zip file
    river_shp_url = f"zip+{river_link}"
    
    # Read shapefile
    with fiona.Env(session=fiona_session):
        SWOT_HR_shps.append(gpd.read_file(river_shp_url)) 

In [ ]:
# Combine granules from time steps into one dataframe
SWOT_HR_df = gpd.GeoDataFrame(pd.concat(SWOT_HR_shps, ignore_index=True))

# Sort dataframe by reach_id
SWOT_HR_df = SWOT_HR_df.sort_values(['reach_id', 'time'])

SWOT_HR_df.explore()